In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import models, layers


2023-09-13 17:44:24.026616: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 17:44:24.061849: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-13 17:44:24.062689: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-13 17:44:24.678510: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [11]:
num_threads = 5
os.environ["OMP_NUM_THREADS"] = "5"
os.environ["TF_NUM_INTRAOP_THREADS"] = "5"
os.environ["TF_NUM_INTEROP_THREADS"] = "5"
tf.config.threading.set_inter_op_parallelism_threads(num_threads)
tf.config.threading.set_intra_op_parallelism_threads(num_threads)
tf.config.set_soft_device_placement(True)

In [12]:
dataset = pd.read_csv('./data/recipe-dataset.csv')
texts = dataset['text'].astype(str)
labels = dataset['label']

In [14]:
max_words = 500
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
data = pad_sequences(sequences, maxlen=max_words) 

In [15]:
data

array([[  0,   0,   0, ...,  70, 131,  22],
       [  0,   0,   0, ...,  15,  64,  22],
       [  0,   0,   0, ...,  12,  17,  69],
       ...,
       [  0,   0,   0, ...,  44,  50,  25],
       [  0,   0,   0, ...,   7,  10, 230],
       [  0,   0,   0, ...,  13, 394,   2]], dtype=int32)

In [16]:
targets = np.array(labels).astype("float32")

In [17]:
test_x = data[:1500] 
test_y = targets[:1500]
train_x = data[1500:]
train_y = targets[1500:]

In [18]:
train_x.shape, train_y.shape

((1828, 500), (1828,))

In [19]:
len(texts[0]), len(sequences[0]), len(data[0])

(959, 134, 500)

In [20]:
model = models.Sequential()
model.add(layers.Dense(50, activation="relu", input_shape=(max_words,)))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                25050     
                                                                 
 dropout (Dropout)           (None, 50)                0         
                                                                 
 dense_1 (Dense)             (None, 50)                2550      
                                                                 
 dropout_1 (Dropout)         (None, 50)                0         
                                                                 
 dense_2 (Dense)             (None, 50)                2550      
                                                                 
 dense_3 (Dense)             (None, 1)                 51        
                                                                 
Total params: 30201 (117.97 KB)
Trainable params: 30201 

2023-09-13 18:04:08.607242: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-13 18:04:08.608321: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [64]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
results = model.fit(train_x, train_y, epochs=5, batch_size=32, validation_data=(test_x, test_y))

Epoch 1/5
58/58 [==============================] - 1s 4ms/step - loss: 1.4569 - accuracy: 0.7609 - val_loss: 0.3527 - val_accuracy: 0.9000
Epoch 2/5
58/58 [==============================] - 0s 2ms/step - loss: 0.6537 - accuracy: 0.8518 - val_loss: 0.2527 - val_accuracy: 0.9120
Epoch 3/5
58/58 [==============================] - 0s 2ms/step - loss: 0.4620 - accuracy: 0.8638 - val_loss: 0.2369 - val_accuracy: 0.9113
Epoch 4/5
58/58 [==============================] - 0s 2ms/step - loss: 0.3676 - accuracy: 0.8846 - val_loss: 0.2247 - val_accuracy: 0.9207
Epoch 5/5
58/58 [==============================] - 0s 2ms/step - loss: 0.3212 - accuracy: 0.8961 - val_loss: 0.2275 - val_accuracy: 0.9140


In [65]:
scores = model.evaluate(test_x, test_y, verbose=0)
print("Accuracy: %.2f%%" % (scores[1] * 100))

Accuracy: 91.40%


In [66]:
model.save('./data/keras-recipe-sentiment.mdl')
loaded_model = models.load_model('./data/keras-recipe-sentiment.mdl')
scores = loaded_model.evaluate(test_x, test_y, verbose=0)
print("Loaded Model Accuracy: %.2f%%" % (scores[1] * 100))

INFO:tensorflow:Assets written to: ./data/keras-recipe-sentiment.mdl/assets


INFO:tensorflow:Assets written to: ./data/keras-recipe-sentiment.mdl/assets


Loaded Model Accuracy: 91.40%


In [94]:
input_text = """
4 medium sweet potatoes
1 tablespoon unsalted butter, melted
1 teaspoon olive oil
1 teaspoon finely chopped fresh thyme leaves
1 garlic clove, finely grated on a microplane
Kosher salt and freshly ground black pepper
1/3 cup nonfat Greek-style yogurt
1 scallion, white and green parts chopped
"""

input_sequence = tokenizer.texts_to_sequences([input_text])
input_data = pad_sequences(input_sequence, maxlen=max_words)
prediction = loaded_model.predict(input_data)
print(prediction)

1/1 [==============================] - 0s 13ms/step
[[1.3444057e-05]]


In [58]:
np.mean([len(t) for t in texts])

387.8722956730769

In [ ]:
from gensim.models import KeyedVectors
import gensim.downloader as api

# Load vectors directly from the file
# model = KeyedVectors.load_word2vec_format('path/to/GoogleNews-vectors-negative300.bin', binary=True)
model = api.load('word2vec-google-news-300')

from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

import numpy as np

nb_words = 500
embed_size = 500

word_index = tokenizer.word_index
nb_words = min(max_words, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))

for word, i in word_index.items():
    if i >= max_words:
        continue
    if word in model.vocab:
        embedding_matrix[i] = model.word_vec(word)

from keras.layers import Embedding

embedding_layer = Embedding(nb_words,
                            embed_size,
                            weights=[embedding_matrix],
                            trainable=False)


In [ ]:
model = models.Sequential()
model.add(embedding_layer)
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation="relu"))
model.add(layers.Dense(1, activation="sigmoid"))
model.summary()